# Clase Estrategia_ MAs

In [1]:
#Librerías a utilizar
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import numpy as np

In [2]:
#Definir Ticker
Ticker='AMZN'

#Definir la comision de Compra / Venta
Comision=0.0025 #Comisión de 0.25% por compra o venta de casa de bolsa

In [3]:
#Definición de funciones a utilizar

#Función de extracción de información de Yahoo Finance de 1 mes cada 60 minutos
def get_data(symbol: str):
    data=yf.download(tickers=symbol, period='44d', interval='60m')
    data.reset_index(inplace=True)
    return data

# Función para calcular el Promedio Móvil
def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Función para determinar la tendencia - equilibrada
def determine_trend(data):
    if data['SMA_10']>data['SMA_20']>data['SMA_30']:
        return 2 # Tendencia de Alza
    elif data['SMA_10']<data['SMA_20']<data['SMA_30']:
        return 1 # Tendencia de Baja
    else:
        return 0 # Sin tendencia definida
    
    # Funcion para Calcular el RSI (Relative Strength Index) 
    # Se utiliza en los mercados como un indicador de Sobrecompra o Sobreventa 
    # Valores Por arriba del 70% indican que esta en un territorio de sobrecompra 
    # Valores Por abajo  del 30% indican que esta en un territorio de sobreventa   
def rsi(DF,n=20):
    df=DF.copy()
    df['dif']=df['Close']-df['Close'].shift(1)
    df['gain']=np.where(df['dif']>=0,df['dif'],0)
    df['loss']=np.where(df['dif']<0,abs(df['dif']),0)
    average_gain=[]
    average_loss=[]
    gain=df['gain'].tolist()
    loss=df['loss'].tolist()
    for i in range(len(df)):
        if i < n:
            average_gain.append(np.NaN)
            average_loss.append(np.NaN)
        elif i==n:
            average_gain.append(df['gain'].rolling(n).mean()[n])
            average_loss.append(df['loss'].rolling(n).mean()[n])
        elif i > n:
            average_gain.append(((n-1)*average_gain[i-1]+gain[i])/n)
            average_loss.append(((n-1)*average_loss[i-1]+loss[i])/n)
    df['average_gain']=np.array(average_gain)
    df['average_loss']=np.array(average_loss)
    df['RS']=df['average_gain']/df['average_loss']
    df['RSI']=100-(100/(1+df['RS']))
    return df['RSI']


In [4]:
#Obtener la información de Yahoo Finance y renombrar columnas
data=get_data(Ticker)
data=data.set_axis(['Datetime','Close','High','Low','Open','Volume'], axis=1)

#Calcular los promedios moviles para los ultimos 10, 20 y 30  valores de la columa 'Close' y se agregan al data frame
data['SMA_10']=calculate_sma(data,10)
data['SMA_20']=calculate_sma(data,20)
data['SMA_30']=calculate_sma(data,30)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
# Determinar la tendencia en una nueva columna del dataframe
data['Tendencia']=data.apply(determine_trend,axis=1)

In [6]:
# Se calcula el RSI y se integra al data frame
RSI_data=rsi(data,n=20)
data=pd.concat([data,RSI_data], axis=1)
data

,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI
0,2025-01-08 14:30:00+00:00,220.679993,223.164993,220.199997,223.149994,5235909,NaN,NaN,NaN,0,NaN
1,2025-01-08 15:30:00+00:00,221.529999,222.145004,220.660004,220.660004,2143742,NaN,NaN,NaN,0,NaN
2,2025-01-08 16:30:00+00:00,220.559998,222.740005,220.440094,221.524994,1877635,NaN,NaN,NaN,0,NaN
3,2025-01-08 17:30:00+00:00,222.910004,223.110001,220.320007,220.580002,1738913,NaN,NaN,NaN,0,NaN
4,2025-01-08 18:30:00+00:00,223.350006,223.520004,221.639999,222.889999,2218310,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
303,2025-03-13 15:30:00+00:00,193.710007,195.500000,193.494995,194.831207,4338810,197.572841,196.525206,196.310870,2,41.314580
304,2025-03-13 16:30:00+00:00,192.539993,194.429993,191.820007,193.679993,4723846,196.967841,196.534921,196.244704,2,39.661708
305,2025-03-13 17:30:00+00:00,193.899994,193.960007,191.910004,192.500000,3869215,196.679340,196.575920,196.244037,2,42.477486
306,2025-03-13 18:30:00+00:00,193.059998,194.559998,192.880005,193.910004,3308906,196.033339,196.495920,196.107367,0,41.226656


In [7]:
# Determinar en que periodos hubo tendencia de alza y de baja - se guarda en un nuevo dataframe y lo reindexamos
df=data[data['Tendencia']!=0]
df=df.reset_index()
df

,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI
0,34,2025-01-15 20:30:00+00:00,223.339996,223.500000,222.720001,223.110001,2904071,221.125398,219.755494,219.602536,2,56.390601
1,35,2025-01-16 14:30:00+00:00,223.122101,224.649994,222.529999,224.559998,4892314,221.522609,220.059599,219.647273,2,55.806337
2,36,2025-01-16 15:30:00+00:00,223.964996,224.509995,222.779999,223.139999,2722173,222.226408,220.363329,219.705772,2,57.595367
3,37,2025-01-16 16:30:00+00:00,222.513504,224.139999,222.070007,223.949997,1932913,222.699759,220.541004,219.846056,2,53.657969
4,38,2025-01-16 17:30:00+00:00,221.389999,222.619995,221.350006,222.529999,1865306,222.528259,220.698004,219.966389,2,50.826890
...,...,...,...,...,...,...,...,...,...,...,...,...
186,292,2025-03-11 18:30:00+00:00,198.345001,199.949997,196.679993,197.229996,6021045,195.145761,195.826135,198.428604,1,48.051460
187,293,2025-03-11 19:30:00+00:00,196.500000,198.600006,196.500000,198.360001,3854296,195.477571,195.679885,198.023604,1,45.370343
188,303,2025-03-13 15:30:00+00:00,193.710007,195.500000,193.494995,194.831207,4338810,197.572841,196.525206,196.310870,2,41.314580
189,304,2025-03-13 16:30:00+00:00,192.539993,194.429993,191.820007,193.679993,4723846,196.967841,196.534921,196.244704,2,39.661708


In [8]:
# Determinar la señal de Compra o Venta y definir el precio de dicha señal
df.loc[0,'Señal']=df.loc[0,'Tendencia']
for i in range(1,len(df)):
    if df.loc[i,'Tendencia']==df.loc[i-1,'Tendencia']:
        df.loc[i,'Señal']=0
    else:
        df.loc[i,'Señal']=df.loc[i,'Tendencia']

In [9]:
#Corregir la primera fila, en caso de que la primera señal, sea una señal de venta
if df.loc[0,'Señal']==1:
    df.loc[0,'Señal']=0
else:
    df.loc[0,'Señal']=2

In [10]:
#Quitar todos los ceros del dataframe con la columna 'Señal'
df=df[df['Señal']!=0]

#Reindexar finalmente el dataframe
df_Final=df.reset_index()
df_Final

,level_0,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI,Señal
0,0,34,2025-01-15 20:30:00+00:00,223.339996,223.500000,222.720001,223.110001,2904071,221.125398,219.755494,219.602536,2,56.390601,2.0
1,57,109,2025-01-31 18:30:00+00:00,238.514999,239.634995,237.949997,239.570007,2684278,236.954651,237.039851,237.150104,1,56.713461,1.0
2,58,119,2025-02-04 14:30:00+00:00,241.440002,241.830002,238.039993,238.925003,7075973,237.889380,237.422015,237.323028,2,61.013637,2.0
3,70,140,2025-02-07 14:30:00+00:00,229.672501,234.809906,228.500000,232.740005,29094795,236.724171,237.956786,238.109310,1,35.154154,1.0
4,188,303,2025-03-13 15:30:00+00:00,193.710007,195.500000,193.494995,194.831207,4338810,197.572841,196.525206,196.310870,2,41.314580,2.0


In [11]:
#Calcular el rendimiento de la estrategia
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Señal']==1:
        df_Final.loc[j,'Rendimiento']=df_Final.loc[j,'Close']*(1-Comision*1.16)/(df_Final.loc[j-1,'Close']*(1+Comision*1.16))-1
    else:
        df_Final.loc[j,'Rendimiento']=0

In [12]:
#Renombrar la columna 'Señal'
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Tendencia']==2:
       df_Final.loc[j,'Señal']= "Compra"
    else:
        df_Final.loc[j,'Señal']="Venta"

C:\Users\luist\AppData\Local\Temp\ipykernel_15488\3650655291.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Compra' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_Final.loc[j,'Señal']= "Compra"


In [13]:
#Generar un listado de columnas para mi resumen del Dataframe
colunmas=['index','Datetime','Close','Señal','RSI','Rendimiento']

#Seleccionar las columnas del dataframe 'df_Final'
df_Resumen=df_Final[colunmas]
df_Resumen


,index,Datetime,Close,Señal,RSI,Rendimiento
0,34,2025-01-15 20:30:00+00:00,223.339996,Compra,56.390601,0.00000
1,109,2025-01-31 18:30:00+00:00,238.514999,Venta,56.713461,0.06177
2,119,2025-02-04 14:30:00+00:00,241.440002,Compra,61.013637,0.00000
3,140,2025-02-07 14:30:00+00:00,229.672501,Venta,35.154154,-0.05424
4,303,2025-03-13 15:30:00+00:00,193.710007,Compra,41.314580,0.00000


In [14]:
df_Stats=df_Resumen[df_Resumen['Rendimiento']!=0]
df_Stats['Rendimiento'].describe().round(4)

count    2.0000
mean     0.0038
std      0.0820
min     -0.0542
25%     -0.0252
50%      0.0038
75%      0.0328
max      0.0618
Name: Rendimiento, dtype: float64

In [15]:
#Crear una columna que me señale la Compra y a que precio
for j in range (0,len(df_Resumen)):
    if df_Resumen.loc[j,'Señal']=='Compra':
        df_Resumen.loc[j,'Señal_Compra']=df_Resumen.loc[j,'Close']
    else:
        df_Resumen.loc[j,'Señal_Compra']=""
              #Crear Una Columna que me diga la compra y a que precio
for j in range(0,len(df_Resumen)):
    if df_Resumen.loc[j,'Señal']=="Venta":
        df_Resumen.loc[j,'Señal_Venta']=df_Resumen.loc[j,'Close']
    else:
        df_Resumen.loc[j,'Señal_Venta']=""

C:\Users\luist\AppData\Local\Temp\ipykernel_15488\1747222132.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Resumen.loc[j,'Señal_Compra']=df_Resumen.loc[j,'Close']
C:\Users\luist\AppData\Local\Temp\ipykernel_15488\1747222132.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_Resumen.loc[j,'Señal_Compra']=""
C:\Users\luist\AppData\Local\Temp\ipykernel_15488\1747222132.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [16]:
# Grafico de Velas Japonesas Y Marcadores de Compra y Venta
grafico=data[:]
fig=go.Figure(data=[go.Candlestick(x=grafico.index,
                                    open=grafico['Open'],
                                    high=grafico['High'],
                                    low=grafico['Low'],
                                    close=grafico['Close'])])
#Agregamos los Promedios Moviles
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_10'],mode='lines',name='SMA 10',line=dict(color='blue')))
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_20'],mode='lines',name='SMA 20',line=dict(color='red')))
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_30'],mode='lines',name='SMA 30',line=dict(color='green')))

# Agregramos marcas de compra y venta 
fig.add_scatter(x=df_Resumen['index'],y=df_Resumen['Señal_Compra'],mode='markers',marker=dict(size=8,color='Black'),name='Compra')
fig.add_scatter(x=df_Resumen['index'],y=df_Resumen['Señal_Venta'],mode='markers',marker=dict(size=8,color='Yellow'),name='Venta')

fig.show()